In [1]:
import numpy as np
import rasterio as rio

In [2]:
# proj crs NAD 1983 UTM Zone 18N
# geographic crs NAD 1983

with rio.open('data_github/030202_201809190900_depth_map_projUTM18n.tif') as ds_in:
    riv_inun = ds_in.read(1, masked=True)
    riv_inun[riv_inun>3.4e+38] = np.nan # set nodata
    riv_inun[riv_inun<0] = 0
    riv_inun_profile = ds_in.profile

with rio.open('data_github/florence_surge_max_compr_lzw.tif') as ds_in:
    surge_inun = ds_in.read(1, masked=True)
    surge_inun[surge_inun < 0] = 0
    surge_inun_profile = ds_in.profile

# give riv_inun one extra row and col to match shapes with surge_inun
newrow = np.empty(riv_inun.shape[1])
newrow.fill(np.nan)
riv_inun = np.vstack([riv_inun,newrow])

newcol = np.empty([riv_inun.shape[0],1])
newcol.fill(np.nan)
riv_inun = np.hstack([riv_inun,newcol])

In [3]:
ratio_adj = 0.001 # 0.001 m = 0.04 in
ratio_array = (surge_inun + ratio_adj) / (riv_inun + ratio_adj)

In [4]:
compound_inun = np.where(ratio_array>1,surge_inun,
                   np.where(ratio_array<0.5,riv_inun,
                      ((np.maximum(surge_inun,riv_inun) + (surge_inun+riv_inun))/2)
                         ))

In [5]:
# https://rasterio.readthedocs.io/en/latest/topics/masks.html
msk = (~surge_inun.mask * 255).astype('uint8')
surge_inun_profile['compress'] = 'lzw'
with rio.open('data_github/compound_inun.tif', 'w', **surge_inun_profile) as ds_out:
    ds_out.write(compound_inun,1)
    ds_out.write_mask(msk)